In [1]:
import glob
import pandas as pd
import numpy as np
import seaborn as sns

cm = sns.light_palette("grey", as_cmap=True)

## 🧱 Sample dataset
Here is what a sample of data looks like 

In [2]:
path =r'../input/global-top-250-retailers/' 
allFiles = glob.glob(path + "/cos*.csv") 
allFiles = np.sort(allFiles)

frame = pd.DataFrame()
list_ = []
cols = [ 'name', 
        'country_of_origin', 
        'retail_revenue', 
        'dominant_operational_format', 
        'countries_of_operation', 
        'retail_revenue_cagr'
       ]
i = 1

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, sep=";", usecols=cols, verbose=False) 
    df["year"] = 2016+i
    list_.append(df)
    del df
    i = i + 1

df = pd.concat(list_)
del list_

#df = df[df["year"].isin([2018, 2019])]
df.sample(10)

,name,country_of_origin,retail_revenue,dominant_operational_format,countries_of_operation,retail_revenue_cagr,year
110,"Canadian Tire Corporation, Limited",Canada,8635.0,Other Specialty,1,0.041,2018
160,Edion Corporation,Japan,6191.0,Electronics Specialty,1,0.000,2019
15,Auchan Holding SA,France,59050.0,Hypermarket/Supercenter/Superstore,14,0.050,2017
86,Dairy Farm International Holdings,Hong Kong,11289.0,Supermarket,11,0.029,2019
31,LVMH Moët Hennessy-Louis Vuitton S.A.,France,33289.0,Other Specialty,70,0.118,2019
119,Colruyt Group,Belgium,7894.0,Supermarket,3,0.050,2017
141,Bic Camera Inc.,Japan,6745.0,Electronics Specialty,1,0.055,2017
70,X5 Retail Group N.V.,Russia,13378.0,Discount Store,1,0.187,2017
171,Coop Danmark A/S,Denmark,5668.0,Supermarket,2,-0.009,2019
224,HORNBACH Baumarkt AG Group,Germany,4083.0,Home Improvement,9,0.043,2018


## 🗺️ From which countries do the top players originate?

In [3]:
table_country = pd.pivot_table(df, 
                               values='retail_revenue', 
                               index=['country_of_origin'], 
                               columns=['year'], 
                               aggfunc=np.sum
                              )
table_country = table_country.reset_index()
table_country.columns = ["country_of_origin", "revenue_2017", "revenue_2018", "revenue_2019"]
sum_2019 = table_country['revenue_2019'].sum()
table_country = table_country.sort_values(by="revenue_2019", ascending=False).head(10)

table_country

,country_of_origin,revenue_2017,revenue_2018,revenue_2019
39,US,1959341.0,2016257.0,2174045.0
14,Germany,413241.0,424993.0,469384.0
19,Japan,280095.0,316845.0,314434.0
13,France,354259.0,348772.0,290637.0
38,UK,249289.0,214754.0,229161.0
9,China,114259.0,118997.0,138804.0
21,Netherlands,93269.0,120780.0,127826.0
0,Australia,86045.0,92703.0,96950.0
7,Canada,84483.0,85172.0,88582.0
31,Spain,72937.0,76395.0,82838.0


In [4]:
print(f"In 2019 ${np.round(sum_2019.astype(int)/1e6, 1)} Trillion revenue was generated by top 250 retailers; {np.round(100*table_country['revenue_2019'].sum()/sum_2019)}% of these originate from {len(table_country)} countries")

In 2019 $4.6 Trillion revenue was generated by top 250 retailers; 87.0% of these originate from 10 countries


## 🇺🇸 Deep dive into firms originating from US 

In [5]:
df_us = df[df["country_of_origin"].isin(["US"]) & (df["year"]==2019)]
cols = ['name', 
        'retail_revenue',
        'dominant_operational_format', 
        'countries_of_operation',
        'retail_revenue_cagr']
df_us = df_us[cols].sort_values(by="retail_revenue", ascending=False).reset_index(drop=True)

df_us.head(10)

,name,retail_revenue,dominant_operational_format,countries_of_operation,retail_revenue_cagr
0,"Wal-Mart Stores, Inc.",500343.0,Hypermarket/Supercenter/Superstore,29,0.013
1,Costco Wholesale Corporation,129025.0,Cash & Carry/Warehouse Club,12,0.054
2,The Kroger Co.,118982.0,Supermarket,1,0.042
3,"Amazon.com, Inc.",118573.0,Non-Store,14,0.180
4,"The Home Depot, Inc.",100904.0,Home Improvement,4,0.062
5,"Hy-Vee, Inc.",100000.0,Supermarket,1,0.054
6,"Walgreens Boots Alliance, Inc.",99115.0,Drug Store/Pharmacy,10,0.067
7,CVS Health Corporation,79398.0,Drug Store/Pharmacy,3,0.045
8,Target Corporation,71879.0,Discount Department Store,1,0.000
9,"Lowe's Companies, Inc.",68619.0,Home Improvement,3,0.063


In [6]:
agg_dict = {'retail_revenue':['sum', 'median'], 
            'retail_revenue_cagr':['median'],
            'name': lambda x: len(x.unique()),
           }

us_agg = df_us.groupby(["dominant_operational_format"]).agg(agg_dict).reset_index()
us_agg.columns = [' '.join(col).strip() for col in us_agg.columns.values]
us_agg.columns = us_agg.columns.str.replace(' ','_')
us_agg.rename({us_agg.columns[-1]: "n_firms"}, axis=1, inplace=True)

us_agg = us_agg.sort_values(by="retail_revenue_sum", ascending=False).reset_index(drop=True)

us_agg["retail_revenue_sum"] /= 1e3
us_agg["retail_revenue_median"] /= 1e3

rename_dict = {
    "dominant_operational_format": "Retail format",
    "retail_revenue_sum": "Total revenue",
    "retail_revenue_median": "Median revenue",
    "retail_revenue_cagr_median": "Median CAGR",
    "n_firms": "# Firms"
}
format_dict = {
    'Total revenue':'${0:,.0f}', 
    'Median revenue':'${0:,.0f}',
    'Median CAGR': '{:.1%}'          
}

#us_agg
(
    us_agg.rename(rename_dict, axis=1)
    .style
    .bar(subset=['Median CAGR'], align='mid', color=['#d65f5f', '#5fba7d'])
    .background_gradient(subset=['Total revenue', '# Firms'], cmap=cm)
    .format(format_dict).hide_index()
    .bar(color='#5fba7d', vmin=0, subset=['Median revenue'], align='zero')
    .set_caption('2019 United States top retailers ($Billion)')
)

Retail format,Total revenue,Median revenue,Median CAGR,# Firms
Hypermarket/Supercenter/Superstore,$519,$260,3.3%,2
Supermarket,$419,$9,4.5%,17
Drug Store/Pharmacy,$194,$79,4.5%,3
Home Improvement,$189,$39,6.2%,4
Cash & Carry/Warehouse Club,$146,$13,5.4%,3
Other Specialty,$144,$6,5.2%,21
Non-Store,$139,$8,11.8%,4
Department Store,$110,$13,-0.2%,9
Apparel/Footwear Specialty,$107,$12,5.7%,8
Electronics Specialty,$73,$36,3.4%,2


## 🛒 2019 Global top retailers

In [7]:
agg_dict = {'retail_revenue':['sum', 'median'], 
            'retail_revenue_cagr':['median'],
            'name': lambda x: len(x.unique()),
           }

us_agg = df[df["year"]==2019].groupby(["dominant_operational_format"]).agg(agg_dict).reset_index()
us_agg.columns = [' '.join(col).strip() for col in us_agg.columns.values]
us_agg.columns = us_agg.columns.str.replace(' ','_')
us_agg.rename({us_agg.columns[-1]: "n_firms"}, axis=1, inplace=True)

us_agg = us_agg.sort_values(by="retail_revenue_sum", ascending=False).reset_index(drop=True)

us_agg["retail_revenue_sum"] /= 1e3
us_agg["retail_revenue_median"] /= 1e3

rename_dict = {
    "dominant_operational_format": "Retail format",
    "retail_revenue_sum": "Total revenue",
    "retail_revenue_median": "Median revenue",
    "retail_revenue_cagr_median": "Median CAGR",
    "n_firms": "# Firms"
}
format_dict = {
    'Total revenue':'${0:,.0f}', 
    'Median revenue':'${0:,.0f}',
    'Median CAGR': '{:.1%}'          
}

#us_agg
(
    us_agg.rename(rename_dict, axis=1)
    .style
    .bar(subset=['Median CAGR'], align='mid', color=['#d65f5f', '#5fba7d'])
    .background_gradient(subset=['Total revenue', '# Firms'], cmap=cm)
    .format(format_dict).hide_index()
    .bar(color='#5fba7d', vmin=0, subset=['Median revenue'], align='zero')
    .set_caption('2019 Global top retailers ($Billion)')
)

Retail format,Total revenue,Median revenue,Median CAGR,# Firms
Supermarket,"$1,081",$9,3.7%,60
Hypermarket/Supercenter/Superstore,"$1,026",$9,3.7%,26
Discount Store,$381,$9,7.5%,18
Other Specialty,$302,$6,6.9%,37
Drug Store/Pharmacy,$274,$6,6.3%,14
Home Improvement,$247,$10,5.7%,10
Apparel/Footwear Specialty,$229,$9,6.7%,19
Non-Store,$211,$11,18.0%,7
Department Store,$208,$8,1.4%,21
Electronics Specialty,$199,$11,2.9%,13
